In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from splink import Splink
from splink.jar_fallback import jw_sim_py


In [2]:
print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("Splink") \
    .master("local[*]") \
    .getOrCreate()

print(spark.sparkContext.appName)

df = spark.read.option("header", "true").csv("/home/abhishekks446/data/output/10k_data.csv")
df_new=df

PySpark Application Started


22/10/07 10:59:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Splink
the data is:


In [3]:
df_new.printSchema()

root
 |-- unique_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- store_number: string (nullable = true)
 |-- store_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- store_location: string (nullable = true)
 |-- county_number: string (nullable = true)
 |-- county: string (nullable = true)
 |-- category: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- vendor_number: string (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- item_number: string (nullable = true)
 |-- item_description: string (nullable = true)
 |-- pack: string (nullable = true)
 |-- bottle_volume_ml: string (nullable = true)
 |-- state_bottle_cost: string (nullable = true)
 |-- state_bottle_retail: string (nullable = true)
 |-- bottles_sold: string (nullable = true)
 |-- sale_dollars: string (nullable = true)
 |-- volume_sold_liters: string (nullable = true)
 |

In [4]:

settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.store_name = r.store_name"
    ],
    "comparison_columns": [

        {
            "col_name": "store_name",
            "num_levels": 4,
            "term_frequency_adjustments": True
        },
        {
            "col_name": "address"
        }
        ,{
            "col_name": "zip_code"
        }
    
    ]

}

linker = Splink(settings,df_new, spark)
df_e = linker.get_scored_comparisons()

/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/case_statements.py:20: UserWarning: 

Custom string comparison functions such as jaro_winkler_sim are not available in Spark
Or you did not pass 'spark' (the SparkSession) into 'Model' 
You can import these functions using the scala-udf-similarity-0.0.9.jar provided with Splink.
You will need to add it by correctly configuring your spark config
For example in Spark 2.4.5

from pyspark.sql import SparkSession, types
from pyspark.context import SparkConf, SparkContext
conf.set('spark.driver.extraClassPath', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar') # Not needed in spark 3
conf.set('spark.jars', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar')
spark.udf.registerJavaFunction('jaro_winkler_sim','uk.gov.moj.dash.linkage.JaroWinkle

In [6]:
df_new=df_e.toPandas().sort_values("match_probability", ascending=False)


In [7]:
# df_new.count()
df_new.head(20)

,match_weight,match_probability,unique_id_l,unique_id_r,store_name_l,store_name_r,gamma_store_name,address_l,address_r,gamma_address,zip_code_l,zip_code_r,gamma_zip_code
0,84.202232,1.0,INV-38604700026,INV-40169000012,Circle K / WDM,Circle K / WDM,3,3501 EP True Parkway,3501 EP True Parkway,1,50265,50265,1
51862,80.394877,1.0,INV-35243100161,INV-42881400252,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51860,80.394877,1.0,INV-35243100161,INV-36321700180,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51859,80.394877,1.0,INV-35243100161,INV-36517000121,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51858,80.394877,1.0,INV-35243100161,INV-43116500032,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51857,80.394877,1.0,INV-35243100161,INV-40476100071,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51856,80.394877,1.0,INV-35243100161,INV-39691400006,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51855,80.394877,1.0,INV-35243100161,INV-40690100130,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51854,80.394877,1.0,INV-35243100161,INV-35592800157,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1
51853,80.394877,1.0,INV-35243100161,INV-38092800168,Hy-Vee Wine and Spirits / Bettendorf,Hy-Vee Wine and Spirits / Bettendorf,3,2890 Devils Glen Road,2890 Devils Glen Road,1,52722,52722,1


In [28]:
dedupe=df_new.loc[df_new['match_probability'] > 0.70]

In [29]:
dedupe.count()


match_weight         193
match_probability    193
unique_id_l          193
unique_id_r          193
title_l              193
title_r              193
gamma_title          193
director_l           193
director_r           193
gamma_director       193
dtype: int64

In [ ]:
# df_e.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output")

In [30]:
dedupe.head(50)

,match_weight,match_probability,unique_id_l,unique_id_r,title_l,title_r,gamma_title,director_l,director_r,gamma_director
3678,1.947533,0.794118,s1636,s2167,A Go! Go! Cory Carson Christmas,A Go! Go! Cory Carson Summer Camp,1,"Stanley Moore, Alex Woo","Stanley Moore, Alex Woo",1
1626,1.947533,0.794118,s3165,s4766,Nothing to Lose 2,Nothing to Lose,2,Alexandre Avancini,Alexandre Avancini,1
2511,1.947533,0.794118,s2214,s4869,The Kissing Booth 2,The Kissing Booth,2,Vince Marcello,Vince Marcello,1
2160,1.947533,0.794118,s5874,s5875,Ip Man,Ip Man 2,1,Wilson Yip,Wilson Yip,1
2159,1.947533,0.794118,s5842,s5875,Ip Man 3,Ip Man 2,2,Wilson Yip,Wilson Yip,1
2158,1.947533,0.794118,s5842,s5874,Ip Man 3,Ip Man,1,Wilson Yip,Wilson Yip,1
302,1.947533,0.794118,s7231,s7232,Krish Trish and Baltiboy: Face Your Fears,Krish Trish and Baltiboy: Oversmartness Never ...,1,Tilak Shetty,Tilak Shetty,1
1234,1.947533,0.794118,s7880,s7884,Rocky,Rocky V,1,John G. Avildsen,John G. Avildsen,1
300,1.947533,0.794118,s7230,s7231,Krish Trish and Baltiboy: Comics of India,Krish Trish and Baltiboy: Face Your Fears,1,Tilak Shetty,Tilak Shetty,1
1229,1.947533,0.794118,s603,s604,The Karate Kid Part II,The Karate Kid Part III,2,John G. Avildsen,John G. Avildsen,1


In [ ]:
from splink.diagnostics import splink_score_histogram
splink_score_histogram(df_e, spark, 100, "match_probability", symmetric=True)

In [ ]:
model.bayes_factor_chart()

df_e.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output")

In [ ]:
model = linker.model
model.probability_distribution_chart()

In [ ]:
from splink.intuition import bayes_factor_intuition_chart
sample = df_e.toPandas().sample(1)
row_dict = sample.to_dict(orient="records")[0]
display(sample)
bayes_factor_intuition_chart(row_dict, model)

In [ ]:
cols = ['unique_id', 'director']
dfpd_l = df.toPandas().sample(300)[cols]
dfpd_l["join_col"] = 1
dfpd_r = dfpd_l.copy()
labels = dfpd_l.merge(dfpd_r, on = "join_col", suffixes=('_l', '_r'))
labels = labels[labels["unique_id_r"]< labels["unique_id_l"]]
labels["clerical_match_score"] = (labels["director_l"] == labels["director_r"]).astype(float)
labels = labels.drop(["director_l", "director_r", "join_col"], axis=1)
labels.head()

In [ ]:
from splink.truth import labels_with_splink_scores, roc_chart, precision_recall_chart
labels_sp = spark.createDataFrame(labels)
labels_and_scores = labels_with_splink_scores(labels_sp, df_e, "unique_id", spark)
labels_and_scores.show()

In [ ]:
labels_and_scores_pd = labels_and_scores.toPandas()
f1 = labels_and_scores_pd["clerical_match_score"] == 1.0
labels_and_scores_pd[f1].sample()

In [ ]:
from splink.cluster import clusters_at_thresholds

thresholds = {"cluster_0_5": 0.5, "cluster_0_9": 0.9}
clusters_at_thresholds(df, df_e, thresholds, linker.model).toPandas()